In [ ]:
import os
import openpyxl
import pdfplumber
import pandas as pd
import numpy as np

# 指定PDF文件夹路径
path = 'C:\Users\83923\Desktop\Bti\数据\特医食品说明书'

# 获取该路径下所有的文件名称
all_items = os.listdir(path)

# 创建一个新的Excel工作簿
new_wb = openpyxl.Workbook()
sheet = new_wb.active  # 获取活动工作表

# 遍历文件列表
for item in all_items:
    if item.endswith(".pdf"):  # 确保是PDF文件
        pdf_path = os.path.join(path, item)
        with pdfplumber.open(pdf_path) as pdf:
            # 遍历PDF页面
            for page in pdf.pages:
                # 提取页面中的所有表格
                tables = page.extract_tables()

                for table in tables:
                    if table:  # 确保表格不为空
                        for row in table:
                            # 确保row是可迭代的并将每个单元格的内容写入Excel
                            if isinstance(row, list):
                                sheet.append(row)  # 这将逐个添加列表中的元素为单独的单元格

        # 格式化输出提取完成的信息
        print(f"{item} 提取完成")

# 将工作簿保存到指定路径
new_wb.save("C:/Users/86151/Desktop/learnpython/pythonProject1/泰迪B/尝试提取1.xlsx")
print("所有文件提取完成，结果保存在 '尝试提取1.xlsx' 中。")

In [81]:
import pdfplumber
import pandas as pd
import numpy as np

def extract_nutrition_table(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        start_extracting = False  # 标记是否开始提取表格
        nutrition_tables = []  # 用于保存营养成分表的所有数据
        table_header = None  # 存储表头，仅在第一次提取表格时使用
        # 获取pdf_file文件名不含后缀不含上级目录
        pdf_file_name = pdf_file.split('/')[-1].split('.')[0]
        
        for page_num, page in enumerate(pdf.pages):
            text = page.extract_text()
            
            # 查找“【营养成分表】”关键字
            if "【营养成分表】" in text and not start_extracting:
                start_extracting = True
                # print(f"在第 {page_num + 1} 页找到标签【营养成分表】")
            

            
            # 提取表格内容
            if start_extracting:
                tables= page.extract_tables()
              
                for table in tables:
                    # 如果是第一页，设置表头
                    
                    if table_header is None:
                        # if "营养成分" in table[0][0]:
                        # 如果 table[0][0] 完全等于 ‘营养成分’
                        if table[0][0] == '营养成分':
                            table_header = table[0]  # 第一行作为表头
                            nutrition_tables.extend(table[1:])  # 添加数据部分
                    else:
                        # 处理跨页表格，跳过表头行（如果存在）
                        if len(table[0]) == len(table_header):
                            if table[0] == table_header:
                                nutrition_tables.extend(table[1:])  # 表头重复，跳过第一行
                            else:
                                nutrition_tables.extend(table)  # 没有表头，直接添加所有行


            # 检查是否需要停止提取（如找到新的标签）
            if start_extracting and "【" in text and "【营养成分表】" not in text:
                # print(f"在第 {page_num + 1} 页找到其他标签，停止提取表格")
                break
        # print(f"共提取到 {len(nutrition_tables)} 行数据")
        # 转换为DataFrame并返回
        if nutrition_tables:
            
            df = pd.DataFrame(nutrition_tables, columns=table_header)
            # 把‘’空白替换成NaN
            df.replace('', np.nan, inplace=True)
                # 检测并合并分割行
            for i in range(len(df) - 1):
               
                if pd.isna(df.iloc[i, 1:]).all():  # 如果当前行除了第一列全是NaN
                    df.iloc[i + 1, 0] = str(df.iloc[i, 0]) + str(df.iloc[i + 1, 0])  # 合并名称
                    df.iloc[i] = np.nan  # 标记当前行为空，以便稍后删除

            df.dropna(how="all", inplace=True)  # 删除被标记为空的行
            return df
        else:
            print(pdf_file_name)
            print("未找到营养成分表。")
            return None

In [69]:
df_nutrition_table_temp = extract_nutrition_table('数据\特医食品说明书\国食注字TY20180002.pdf')
df_nutrition_table_temp

在第 1 页找到标签【营养成分表】
在第 2 页找到其他标签，停止提取表格


,营养成分,每100mL,每100kJ
0,能量(kJ),43,100
1,蛋白质(g),0,0
2,脂肪(g),0,0
3,碳水化合物(g),2.5,5.8
4,钠(mg),115,271
5,钾(mg),78,184
6,镁(mg),2.4,5.6
7,磷(mg),6.2,14.6
8,氯(mg),177,416


In [82]:
import os
import openpyxl
import pdfplumber

# 指定PDF文件夹路径
path = '数据\特医食品说明书'

# 获取该路径下所有的文件名称
all_items = os.listdir(path)

# 定义营养成分列表
nutrients = ["能量(kJ)", "脂肪(g)", "碳水化合物(g)", "蛋白质(g)", "钠(mg)", "氯(mg)", "钾(mg)", "磷(mg)"]
# 创建一个空的DataFrame来存储结果
result_df = pd.DataFrame(columns=["注册证号"] + nutrients)
# 遍历文件列表
for item in all_items:
    if item.endswith(".pdf"):  # 确保是PDF文件
        pdf_path = os.path.join(path, item)
        register_no = item.replace('.pdf', '') # 去掉文件扩展名, 作为注册证号
        df_nutrition_table = extract_nutrition_table(pdf_path)  # 提取营养成分表 dataframe
        # df_nutrition_table = df_nutrition_table['营养成分', '每100kJ']  # 保留指定列

        # 初始化营养成分数据
        nutrition_data = {nutrient: 0 for nutrient in nutrients}
         # 遍历营养成分表
        if df_nutrition_table is None:
            continue
        for index, row in df_nutrition_table.iterrows():
            nutrient = row['营养成分']
            if nutrient in nutrition_data:
                nutrition_data[nutrient] = row['每100kJ']

        # 创建一个新的DataFrame行
        row_data = pd.DataFrame([[register_no] + [nutrition_data[nutrient] for nutrient in nutrients]], columns=result_df.columns)
        
        # 使用pd.concat追加数据
        result_df = pd.concat([result_df, row_data], ignore_index=True)


result_df

数据\特医食品说明书\国食注字TY20220013
未找到营养成分表。
数据\特医食品说明书\国食注字TY20230013
未找到营养成分表。
数据\特医食品说明书\国食注字TY20230019
未找到营养成分表。


,注册证号,能量(kJ),脂肪(g),碳水化合物(g),蛋白质(g),钠(mg),氯(mg),钾(mg),磷(mg)
0,国食注字TY20175001,100,1.2,2.5,0,10.1,20.8,28.5,21.3
1,国食注字TY20175002,100,1.28,2.51,0.55,9,15,32,15
2,国食注字TY20175003,100.0,1.280,2.549,0.546,9.441,15.38,31.47,10.49
3,国食注字TY20180001,100,1.3,2.6,0.56,8.4,15.4,0,13.1
4,国食注字TY20180002,100,0,5.8,0,271,416,184,14.6
...,...,...,...,...,...,...,...,...,...
174,国食注字TY20240018,100,0.76,3.15,0.95,23,18,32,11
175,国食注字TY20240019,100,0,0.3,5.6,11,0,0,0
176,国食注字TY20240020,100,2.3,0.69,0.22,3.2,0,0,0
177,国食注字TY20240021,100,0,5.71,0.18,14.17,0,0,0


In [84]:
# 把result_dfb保存到Excel文件,命名为第一题1.xlsx
for col in result_df.columns[1:]:  # 假设第一列是非数值列，可以选择性地排除
    result_df[col] = pd.to_numeric(result_df[col], errors='coerce')
result_df.to_excel('第一题1_1.xlsx', index=False)

 # 蛋白质含量最高的三个

In [87]:
# 读入第一题1_1_完全版.xlsx
df_good_nutrition = pd.read_excel('第一题1_1_完全版.xlsx')
top_3_protein = df_good_nutrition.nlargest(3, '蛋白质(g)')
top_3_protein

,注册证号,能量(kJ),脂肪(g),碳水化合物(g),蛋白质(g),钠(mg),氯(mg),钾(mg),磷(mg)
134,国食注字TY20230045,100,0.04,0.007,5.9,45.0,0.0,0.0,0.0
78,国食注字TY20220002,100,0.00,0.100,5.8,80.4,0.0,0.0,0.0
151,国食注字TY20230062,100,0.00,0.080,5.8,24.0,0.0,0.0,0.0


In [ ]:
# 读入data1.2.xlsx
df_info = pd.read_excel('data1.2.xlsx')

In [89]:
def categorize_population(row):
    if '婴儿' in row['适用人群']:
        return '特医婴配食品'
    else:
        return '1岁以上特医食品'

df_info['适用人群类别'] = df_info.apply(categorize_population, axis=1)
df_info.head()

,企业名称,产品名称,注册证号,有效期至,产品类别,组织状态,适用人群,适用人群类别
0,SHS INTERNATIONAL LTD,纽康特特殊医学用途婴儿氨基酸配方食品,国食注字TY20175001,2027-10-13,氨基酸配方,粉状,食物蛋白过敏婴儿,特医婴配食品
1,ABBOTT LABORATORIES S.A.,雅培亲护特殊医学用途婴儿乳蛋白部分水解配方粉,国食注字TY20175002,2027-10-13,乳蛋白部分水解配方,粉状,0～12月龄乳蛋白过敏高风险婴儿,特医婴配食品
2,ABBOTT LABORATORIES S.A.,菁挚呵护特殊医学用途婴儿乳蛋白部分水解配方粉,国食注字TY20175003,2022-11-19,乳蛋白部分水解配方,粉状,0-12月龄乳蛋白过敏高风险婴儿,特医婴配食品
3,杭州贝因美母婴营养品有限公司,贝因美特殊医学用途婴儿无乳糖配方食品,国食注字TY20180001,2028-01-17,无乳糖配方,粉状,0～12月龄乳糖不耐受婴儿,特医婴配食品
4,SHS INTERNATIONAL LTD,纽贝瑞特殊医学用途婴儿苯丙酮尿症配方粉,国食注字TY20185001,2028-01-17,氨基酸代谢障碍配方,粉状,0～12月龄苯丙酮尿症婴儿,特医婴配食品


In [91]:
df_info.to_excel('第一题1_3.xlsx', index=False)

In [96]:
register_no = '国食注字TY20180002'
register_no[10]

'0'

In [97]:
def extract_info_from_register_no(register_no):
    year = register_no[6:10]
    source = "进口产品" if register_no[10] == '5' else "国产产品"
    return year, source

df_info['登记年份'], df_info['产品来源'] = zip(*df_info['注册证号'].apply(extract_info_from_register_no))
df_info.head()

,企业名称,产品名称,注册证号,有效期至,产品类别,组织状态,适用人群,适用人群类别,登记年份,产品来源
0,SHS INTERNATIONAL LTD,纽康特特殊医学用途婴儿氨基酸配方食品,国食注字TY20175001,2027-10-13,氨基酸配方,粉状,食物蛋白过敏婴儿,特医婴配食品,2017,进口产品
1,ABBOTT LABORATORIES S.A.,雅培亲护特殊医学用途婴儿乳蛋白部分水解配方粉,国食注字TY20175002,2027-10-13,乳蛋白部分水解配方,粉状,0～12月龄乳蛋白过敏高风险婴儿,特医婴配食品,2017,进口产品
2,ABBOTT LABORATORIES S.A.,菁挚呵护特殊医学用途婴儿乳蛋白部分水解配方粉,国食注字TY20175003,2022-11-19,乳蛋白部分水解配方,粉状,0-12月龄乳蛋白过敏高风险婴儿,特医婴配食品,2017,进口产品
3,杭州贝因美母婴营养品有限公司,贝因美特殊医学用途婴儿无乳糖配方食品,国食注字TY20180001,2028-01-17,无乳糖配方,粉状,0～12月龄乳糖不耐受婴儿,特医婴配食品,2018,国产产品
4,SHS INTERNATIONAL LTD,纽贝瑞特殊医学用途婴儿苯丙酮尿症配方粉,国食注字TY20185001,2028-01-17,氨基酸代谢障碍配方,粉状,0～12月龄苯丙酮尿症婴儿,特医婴配食品,2018,进口产品


In [98]:
df_info.to_excel('任务1.4.xlsx', index=False)

In [ ]:
# 选取df_info的登记年份和产品来源两列，为df_2_1
df_2_1 = df_info[['登记年份', '产品来源']]
# df_2_1选取产品来源为国产产品的行，为d
df_2_1_MadeinChina = df_2_1[df_2_1['产品来源'] == '国产产品']

df_2_1_import = df_2_1[df_2_1['产品来源'] == '进口产品']
# df_2_1_import.shape, df_2_1_MadeinChina.shape

((31, 2), (151, 2))

2.1 画图数据

In [ ]:
import pandas as pd
imported_products_count = df_2_1_import.groupby('登记年份').size().reset_index(name='数量')

# 国产

MadeinChina_products_count = df_2_1_MadeinChina.groupby('登记年份').size().reset_index(name='数量')
print(MadeinChina_products_count)
print(imported_products_count)
print(MadeinChina_products_count['数量'])
# 列表形式输出出口产品数量
print(imported_products_count['数量'].tolist())

In [125]:
# df_info选取注册证号，产品来源，适用人群类别三列，为df_2_2
df_2_2 = df_info[['注册证号', '产品来源', '适用人群类别']].copy()
df_2_2.to_excel('任务2.2.xlsx', index=False)

In [127]:
#df_2_2的产品来源和适用人群类别两列分别输出为list
product_sources = df_2_2['产品来源'].tolist()
population_categories = df_2_2['适用人群类别'].tolist()
# print(product_sources)
# print(population_categories)
# 输出product_sources中国产产品个数，进口产品个数
print(product_sources.count('国产产品'))
print(product_sources.count('进口产品'))

151
31


In [128]:
# 统计'适用人群类别'为”特医婴配食品“中，国产产品和进口产品的数量
teyi_yingpei = df_2_2[df_2_2['适用人群类别'] == '特医婴配食品']
teyi_yingpei_counts = teyi_yingpei['产品来源'].value_counts()
print("特医婴配食品中国产产品的数量:", teyi_yingpei_counts.get('国产产品', 0))
print("特医婴配食品中进口产品的数量:", teyi_yingpei_counts.get('进口产品', 0))

# 统计'适用人群类别'为”1岁以上特医食品“中，国产产品和进口产品的数量
one_year_plus = df_2_2[df_2_2['适用人群类别'] == '1岁以上特医食品']
one_year_plus_counts = one_year_plus['产品来源'].value_counts()
print("1岁以上特医食品中国产产品的数量:", one_year_plus_counts.get('国产产品', 0))
print("1岁以上特医食品中进口产品的数量:", one_year_plus_counts.get('进口产品', 0))

特医婴配食品中国产产品的数量: 27
特医婴配食品中进口产品的数量: 24
1岁以上特医食品中国产产品的数量: 124
1岁以上特医食品中进口产品的数量: 7


    { value: 151, name: '国产产品' },
    { value: 31, name: '进口产品' },

In [137]:
# df_info选出 产品名称 适用人群 适用人群类别 组织状态 产品类别 产品来源 
df_3 = df_info[['注册证号','产品名称', '适用人群', '适用人群类别', '组织状态', '产品类别', '产品来源']].copy()
df_3

,注册证号,产品名称,适用人群,适用人群类别,组织状态,产品类别,产品来源
0,国食注字TY20175001,纽康特特殊医学用途婴儿氨基酸配方食品,食物蛋白过敏婴儿,特医婴配食品,粉状,氨基酸配方,进口产品
1,国食注字TY20175002,雅培亲护特殊医学用途婴儿乳蛋白部分水解配方粉,0～12月龄乳蛋白过敏高风险婴儿,特医婴配食品,粉状,乳蛋白部分水解配方,进口产品
2,国食注字TY20175003,菁挚呵护特殊医学用途婴儿乳蛋白部分水解配方粉,0-12月龄乳蛋白过敏高风险婴儿,特医婴配食品,粉状,乳蛋白部分水解配方,进口产品
3,国食注字TY20180001,贝因美特殊医学用途婴儿无乳糖配方食品,0～12月龄乳糖不耐受婴儿,特医婴配食品,粉状,无乳糖配方,国产产品
4,国食注字TY20185001,纽贝瑞特殊医学用途婴儿苯丙酮尿症配方粉,0～12月龄苯丙酮尿症婴儿,特医婴配食品,粉状,氨基酸代谢障碍配方,进口产品
...,...,...,...,...,...,...,...
177,国食注字TY20240017,德瑞清®特殊医学用途流质配方食品,10岁以上需要限制脂肪摄入、消化吸收障碍等医学状况下的人群。,1岁以上特医食品,液态,非全营养配方食品,国产产品
178,国食注字TY20240018,冬泽益全特殊医学用途全营养配方食品,10岁以上进食受限、消化吸收障碍、代谢紊乱等需要补充营养的人群。,1岁以上特医食品,液态,全营养配方食品,国产产品
179,国食注字TY20240019,冬泽力亮特殊医学用途蛋白质组件配方食品,10岁以上特定疾病或医学状况下需要补充蛋白质的人群。,1岁以上特医食品,液态,非全营养配方食品,国产产品
180,国食注字TY20240020,冬泽益能特殊医学用途脂肪组件配方食品,1岁以上特定疾病或者医学状况下需要补充中链脂肪的人群。,1岁以上特医食品,粉状,脂肪（脂肪酸）组件,国产产品


In [139]:
df_3.to_excel('任务3_1更新.xlsx', index=False)

In [ ]:
# 合并 df_3 和 df_good_nutrition，按照 '注册证号' 列进行合并
train_data = pd.merge(df_3, df_good_nutrition, on='注册证号')

# 显示合并后的 DataFrame
train_data .head()

In [143]:
train_data.to_excel('任务3训练数据.xlsx', index=False)

In [144]:
train_data.head()

,注册证号,产品名称,适用人群,适用人群类别,组织状态,产品类别,产品来源,能量(kJ),脂肪(g),碳水化合物(g),蛋白质(g),钠(mg),氯(mg),钾(mg),磷(mg)
0,国食注字TY20175001,纽康特特殊医学用途婴儿氨基酸配方食品,食物蛋白过敏婴儿,特医婴配食品,粉状,氨基酸配方,进口产品,100,1.20,2.500,0.000,10.100,20.80,28.50,21.30
1,国食注字TY20175002,雅培亲护特殊医学用途婴儿乳蛋白部分水解配方粉,0～12月龄乳蛋白过敏高风险婴儿,特医婴配食品,粉状,乳蛋白部分水解配方,进口产品,100,1.28,2.510,0.550,9.000,15.00,32.00,15.00
2,国食注字TY20175003,菁挚呵护特殊医学用途婴儿乳蛋白部分水解配方粉,0-12月龄乳蛋白过敏高风险婴儿,特医婴配食品,粉状,乳蛋白部分水解配方,进口产品,100,1.28,2.549,0.546,9.441,15.38,31.47,10.49
3,国食注字TY20180001,贝因美特殊医学用途婴儿无乳糖配方食品,0～12月龄乳糖不耐受婴儿,特医婴配食品,粉状,无乳糖配方,国产产品,100,1.30,2.600,0.560,8.400,15.40,0.00,13.10
4,国食注字TY20185001,纽贝瑞特殊医学用途婴儿苯丙酮尿症配方粉,0～12月龄苯丙酮尿症婴儿,特医婴配食品,粉状,氨基酸代谢障碍配方,进口产品,100,1.21,0.000,0.000,9.000,19.00,26.00,17.00
